In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("anper.csv")

In [ ]:
#la categoría Libros la pasamos a Libros y cursos, dejando Libros en la subcategoría
df.loc[df.categoria == "Libros", "subcategoria"] = "Libros"
df.loc[df.categoria == "Libros", "categoria"] = 'Libros y cursos'

In [ ]:
#las categorías de Arcillas las ponemos como subcategorías pertenientes a "Arcillas"
df.loc[df.categoria == "Arcilla Alta temperatura", "subcategoria"] = "Arcillas alta temperatura"
df.loc[df.categoria == "Arcilla Alta temperatura", "categoria"] = 'Arcillas'

df.loc[df.categoria == "Arcilla Baja temperatura", "subcategoria"] = "Arcillas alta temperatura"
df.loc[df.categoria == "Arcilla Baja temperatura", "categoria"] = 'Arcillas'

In [ ]:
df.loc[df.categoria == "Baldosas", "subcategoria"] = "Azulejos"
df.loc[df.categoria == "Baldosas", "categoria"] = 'tercer-fuego'
df.loc[df.categoria == "3er Fuego", "categoria"] = 'tercer-fuego'

In [ ]:
df.loc[df.categoria == "Construcción de hornos", "categoria"] = 'Horno'
df.loc[df.categoria == "Materiales para hornear", "subcategoria"] = "Material de estibado para el horno"
df.loc[df.categoria == "Materiales para hornear", "categoria"] = 'Horno'
df.loc[df.categoria == "Hornos", "categoria"] = 'Horno'
df.loc[df.categoria == "Pirometría", "categoria"] = 'Horno'

In [ ]:
df.loc[df.categoria == "Pinceles", "subcategoria"] = "Pincel"
df.loc[df.categoria == "Pinceles", "categoria"] = 'Pinceles/Espátulas pintor'

In [ ]:
df.loc[df.categoria == "Tornos", "subcategoria"] = "Tornos y laminadoras"
df.loc[df.categoria == "Tornos", "categoria"] = 'Maquinaria'

In [ ]:
df.loc[df.categoria == "Elementos protección", "subcategoria"] = "Elementos protección"
df.loc[df.categoria == "Elementos protección", "categoria"] = 'Varios'
df.loc[df.categoria == "Complementos", "categoria"] = 'Varios'

In [ ]:
df.loc[df.categoria == "Moldes y accesorios", "subcategoria"] = "Moldes y accesorios"
df.loc[df.categoria == "Moldes y accesorios", "categoria"] = 'Varios'

In [ ]:
df.loc[df.categoria == "Rotulador Cerámico", "subcategoria"] = "Rotulador Cerámico"
df.loc[df.categoria == "Rotulador Cerámico", "categoria"] = 'Lápices/Tizas cerámicas'

In [ ]:
df.loc[df.categoria == "Cristalizaciones", "subcategoria"] = "Cristalizaciones"
df.loc[df.categoria == "Cristalizaciones", "categoria"] = 'Materias primas'

In [ ]:
#paso valores a tipo de dato categórico
df['categoria'] = df['categoria'].astype('category')
df['tienda'] = df['tienda'].astype('category')

In [ ]:
def procesar_precios(precio):
    if not precio:  # Verifica si la cadena está vacía
        return [('N/A', 0.0)]
    
    # Dividir los valores por '\n'
    items = precio.split('\n')
    result = []
    
    if len(items) > 1:
        for i in range(0, len(items), 2):
            if i+1 < len(items):
                cantidad = items[i]
                try:
                    precio_valor = float(items[i+1].replace('€', '').replace(',', '.'))
                except ValueError:
                    precio_valor = 0.0  # Maneja valores no numéricos o mal formateados
                result.append((cantidad, precio_valor))
    else:
        # Si no tiene el formato esperado, asumir que es un precio único
        try:
            precio_valor = float(precio.replace('€', '').replace(',', '.'))
        except ValueError:
            precio_valor = 0.0  # Maneja valores no numéricos o mal formateados
        result.append(('N/A', precio_valor))  # 'N/A' para indicar que no hay cantidad específica
    
    return result

# Aplicar la función a la columna precio
df['precios_procesados'] = df['precio'].apply(procesar_precios)

# Crear un nuevo DataFrame con las cantidades y precios
filas = []
for idx, row in df.iterrows():
    for cantidad, precio in row['precios_procesados']:
        filas.append({'Cantidad': cantidad, 'Precio': precio})

In [ ]:
df = df.explode('precios_procesados')
df[['Cantidad', 'precio_final']] = pd.DataFrame(df['precios_procesados'].tolist(), index=df.index)

In [ ]:
#si el valor en cantidad no es N/A lo apendeo a la columna nombre
df['nombre'] = df.apply(
    lambda row: f"{row['nombre']} {row['Cantidad']}" if row['Cantidad'] != 'N/A' else row['nombre'], axis=1
)

In [ ]:
df = df.drop(columns=['precio_final', 'Cantidad', 'precios_procesados','descripcion'])

In [ ]:
df.to_csv("anper.csv", index=False)